#### CODE : Entropy 

In [454]:
import numpy as np

In [455]:
def entropy(var):
    N = var.shape[0]
    values, counts = np.unique(var, return_counts=True )
    
    ent = 0.0
    
    for i in counts:
        p = i/N
        ent += (p * np.log2(p))
        
    return -ent

In [456]:
Y = np.array([1,0,0,1,0,1,0,1])

In [457]:
X = np.array([1,1,1,1,1,1,1])

In [458]:
entropy(Y)

1.0

In [459]:
entropy(X)

-0.0

### CODE : Split Data

In [460]:
import pandas as pd

In [461]:
df = pd.read_csv('file.csv')
output_feature='job_satisfaction'

In [462]:
df.head()

,emp_id,department,job_role,over_time,performance,attrition,job_satisfaction
0,1,hr,hr,yes,excellent,yes,very_satisfied
1,2,marketing,marketing,yes,excellent,no,very_satisfied
2,3,finance,finance,no,good,no,satisfied
3,4,it,operation,yes,bad,yes,dissatisfied
4,5,hr,hr,no,excellent,yes,neutral


In [463]:
# test=df.values.tolist();
# print(test)
# for i in range(len(test)):
#     test[i].pop(0)
# print(test)

# df=pd.DataFrame(test);
# df.head(5)
df=pd.read_csv('../temp.csv')
print(df)

   department   job_role over_time performance attrition job_satisfaction
0          hr         hr       yes   excellent       yes   very_satisfied
1   marketing  marketing       yes   excellent        no   very_satisfied
2     finance    finance        no        good        no        satisfied
3          it  operation       yes         bad       yes     dissatisfied
4          hr         hr        no   excellent       yes          neutral
5     finance    finance       yes        good        no     dissatisfied
6          hr         hr        no   excellent       yes        satisfied
7   marketing  marketing       yes         bad        no   very_satisfied
8          hr         hr        no   excellent       yes   very_satisfied
9          it  operation       yes         bad        no        satisfied
10         hr         hr        no   excellent       yes     dissatisfied
11         it  operation       yes        good        no        satisfied
12  marketing  marketing        no    

In [464]:
def divide_data(data, feature):
    # here we are working data frames.
    
    DATA = {}
    
    feat_values = list(data[feature].value_counts().index)
    occurence = list(data[feature].value_counts())
    
    for val in feat_values:
        DATA[val] = {'data' : pd.DataFrame([], columns = data.columns), 'len': 0}
        
    
    for ix in range(data.shape[0]):
        val = data[feature].iloc[ix]
        
        DATA[val]['data'] = DATA[val]['data']._append(data.iloc[ix])
    
        idx = feat_values.index(val)
        DATA[val]['len']  = occurence[idx]
    
    return DATA

In [465]:
divide_data(df,'department')

{'hr': {'data':    department job_role over_time performance attrition job_satisfaction
  0          hr       hr       yes   excellent       yes   very_satisfied
  4          hr       hr        no   excellent       yes          neutral
  6          hr       hr        no   excellent       yes        satisfied
  8          hr       hr        no   excellent       yes   very_satisfied
  10         hr       hr        no   excellent       yes     dissatisfied,
  'len': 5},
 'marketing': {'data':    department   job_role over_time performance attrition job_satisfaction
  1   marketing  marketing       yes   excellent        no   very_satisfied
  7   marketing  marketing       yes         bad        no   very_satisfied
  12  marketing  marketing        no         bad        no        satisfied
  14  marketing  marketing        no         bad        no        satisfied,
  'len': 4},
 'finance': {'data':    department job_role over_time performance attrition job_satisfaction
  2     finance  fin

### CODE : Information Gain

In [466]:
def information_gain(data, feature):
    examples = data.shape[0]
    
    DATA = divide_data(data, feature)
    
    keys = DATA.keys()
    
    
    ent_of_children = 0.0
    
    for key in keys:
        ent_of_children +=  ( (DATA[key]['len']/examples) * entropy(DATA[key]['data'][output_feature]) )
        
    info_gain = entropy(data[output_feature]) - ent_of_children
    return info_gain

In [467]:
information_gain(df, 'department')

0.5316112300691394

In [468]:
information_gain(df, 'job_role')

0.5316112300691394

In [469]:
information_gain(df, 'over_time')

0.19679163198163652

In [470]:
information_gain(df, 'performance')

0.3655962303576019

In [471]:
information_gain(df, 'attrition')

0.2537751634665262

### Constructing a Decision Tree


In [472]:
class DecisionTree:
    
    # constructor
    def __init__(self, depth=0, max_depth=5):
        # Creating a Node
        self.children = {}
        self.fkey = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None
        
    def train(self, data):
        features = data.columns[:-1]
        
        info_gains = []
            
        for f in features:
            i_gain = information_gain(data, f)
            info_gains.append(i_gain)
            
        # finding the best feature
        self.fkey = features[np.argmax(info_gains)]
        
        #Spliting the Data
        DATA = divide_data(data, self.fkey)
        
        
        
        # Giving a target label to the Node
        labels = list(data[output_feature].value_counts().index)
        freq = list(data[output_feature].value_counts().values)
        
        self.target = labels[np.argmax(freq)]
        

        ###### STOPPING CONDITIONS ######
        
        have_data = 0
        keys = DATA.keys()
        
        for key in keys:
            if DATA[key]['len'] > 0:
                have_data +=1
        
        # 1. If it is pure node 
        if have_data<2:
            print("\t"*self.depth + "Leaf Node : "+ self.target)
            return 
        
    
        # 2. Early Stop if you have reached max depth
        if(self.depth >= self.max_depth):
            print("\t"*self.depth + "Leaf Node : "+ self.target)
            return
        
        
        print("\t"*self.depth + "Node : ", self.fkey)
        
        
        # Recursively train child Node
        for key in keys:
            new_data = DATA[key]['data']
            self.children[key] = DecisionTree(depth = self.depth + 1)
            self.children[key].train(new_data)
            
        return  
    
    
    def predict(self, test):
        if self.children == {}:
            return self.target
        print("\t"*self.depth + "Testing Node for - ", self.fkey);
        return self.children[test[self.fkey][0]].predict(test)

# Model

In [473]:
model = DecisionTree()

In [474]:
model.train(df)

Node :  department
	Node :  over_time
		Leaf Node : neutral
		Leaf Node : very_satisfied
	Node :  over_time
		Leaf Node : very_satisfied
		Leaf Node : satisfied
	Node :  over_time
		Leaf Node : dissatisfied
		Leaf Node : satisfied
	Node :  attrition
		Leaf Node : satisfied
		Leaf Node : dissatisfied


In [475]:
model

In [476]:
model.target

'satisfied'

In [477]:
model.fkey

'department'

In [478]:
model.children

{'hr': <__main__.DecisionTree at 0x147683280>,
 'marketing': <__main__.DecisionTree at 0x14766d660>,
 'finance': <__main__.DecisionTree at 0x147383a00>,
 'it': <__main__.DecisionTree at 0x1475c8190>}

In [479]:
model.children['finance']

In [480]:
model.children['it'].fkey

'attrition'

In [481]:
model.children['finance'].children

{'yes': <__main__.DecisionTree at 0x147380610>,
 'no': <__main__.DecisionTree at 0x147380ac0>}

In [482]:
model.children['it'].children

{'no': <__main__.DecisionTree at 0x1475cbe80>,
 'yes': <__main__.DecisionTree at 0x1475ca3b0>}

In [483]:
# model.children['it'].children['attrition'].target

# Prediction 

## Predict 1

In [484]:
test_data=[]
x_test = pd.DataFrame([['hr','hr','yes','excellent','yes']], columns=list(df.columns.values[:-1]))

In [485]:
x_test

,department,job_role,over_time,performance,attrition
0,hr,hr,yes,excellent,yes


In [486]:
model.predict(x_test)

Testing Node for -  department
	Testing Node for -  over_time


'very_satisfied'

## Predict 2 

In [497]:
test_data=[]
x_test = pd.DataFrame([['it','operation','no','good','no']], columns=list(df.columns.values[:-1]))

In [498]:
x_test

,department,job_role,over_time,performance,attrition
0,it,operation,no,good,no


In [499]:
model.predict(x_test)

Testing Node for -  department
	Testing Node for -  attrition


'satisfied'